In [20]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TextEyeTrackingDataset(Dataset):
    def __init__(self, csv_file):
        self.data_frame = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # 假设csv中的列名分别为"text_sequences", "eye_tracking_sequences", "labels"
        text_seq = np.array(eval(self.data_frame.iloc[idx]['text_sequences']))
        eye_tracking_seq = np.array(eval(self.data_frame.iloc[idx]['eye_tracking_sequences']))
        labels = np.array(eval(self.data_frame.iloc[idx]['labels']))

        # 仅使用单词索引作为文本特征的示例，实际应用中应使用词嵌入
        word_indices = text_seq[:, 0]  # 假设单词索引在第0列
        word_indices = [int(i) for i in word_indices]
        word_features = text_seq[:, 2:].astype(np.float32)  # 假设坐标和大小特征从第2列开始

        # 标准化眼动追踪数据（这里仅为示例，实际应用中应根据数据分布进行标准化）
        eye_tracking_features = (eye_tracking_seq - eye_tracking_seq.mean(axis=0)) / eye_tracking_seq.std(axis=0)

        return {
            'word_indices': torch.tensor(word_indices, dtype=torch.long),
            'word_features': torch.tensor(word_features, dtype=torch.float),
            'eye_tracking_features': torch.tensor(eye_tracking_features, dtype=torch.float),
            'labels': torch.tensor(labels, dtype=torch.float)
        }


In [35]:
import torch
import torch.nn as nn

class MultiModalModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(MultiModalModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # 假设单词特征有4维
        self.text_lstm = nn.LSTM(embedding_dim + 4, hidden_dim, batch_first=True)
        # 假设眼动追踪特征有3维
        self.eye_tracking_lstm = nn.LSTM(3, hidden_dim, batch_first=True)
        # 注意这里的输出维度变为hidden_dim，因为我们要对每个时间步进行预测
        self.fc_text = nn.Linear(hidden_dim, output_dim)
        self.fc_eye_tracking = nn.Linear(hidden_dim, output_dim)
        # 最终的分类器不再需要，因为我们已经在每个步骤中进行了分类

    def forward(self, word_indices, word_features, eye_tracking_features):
        word_embeddings = self.embedding(word_indices)
        text_features = torch.cat((word_embeddings, word_features), dim=-1)

        text_out, _ = self.text_lstm(text_features)
        eye_tracking_out, _ = self.eye_tracking_lstm(eye_tracking_features)

        # 对文本LSTM的每个时间步输出进行分类
        text_classifications = self.fc_text(text_out)
        # 对眼动追踪LSTM的每个时间步输出进行分类（如果需要）
        # eye_tracking_classifications = self.fc_eye_tracking(eye_tracking_out)

        # 由于问题描述中只需要对文本序列中的每个单词进行分类，所以我们只返回文本分类结果
        # 如果你的任务需要考虑眼动追踪数据对每个单词的影响，你可能需要设计一个机制来融合这两个分类结果
        return text_classifications


In [42]:
from torch.utils.data import random_split
# 加载数据
dataset = TextEyeTrackingDataset('data/raw_data_0317.csv')
batch_size = 1

# 假设dataset是你已经加载的完整数据集
train_size = int(len(dataset) * 0.8)  # 以80%的数据作为训练集
test_size = len(dataset) - train_size  # 剩余20%的数据作为测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
import numpy as np

def evaluate_model(model, test_loader):
    model.eval()  # 设置模型为评估模式
    predictions = []
    actuals = []
    with torch.no_grad():  # 在评估过程中不计算梯度
        for batch in test_loader:
            outputs = model(batch['word_indices'], batch['word_features'], batch['eye_tracking_features'])
            outputs = torch.sigmoid(outputs.squeeze(dim=2).view(-1))  # 使用sigmoid激活函数获取概率,因为要算准确率
            predictions.extend(outputs.tolist())
            actuals.extend(batch['labels'].view(-1).tolist())

    # 计算性能指标
    predictions = np.round(predictions)  # 将概率值转换为0或1
    accuracy = accuracy_score(actuals, predictions)
    auc = roc_auc_score(actuals, predictions)
    precision = precision_score(actuals, predictions)
    recall = recall_score(actuals, predictions)

    return accuracy, auc, precision, recall

def train_model(model, train_loader, test_loader, optimizer, criterion, epochs=10):
    for epoch in range(epochs):
        model.train()  # 设置模型为训练模式
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            outputs = model(batch['word_indices'], batch['word_features'], batch['eye_tracking_features'])
            outputs = outputs.squeeze(dim=2).view(-1, 1)
            loss = criterion(outputs, batch['labels'].view(-1, 1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # 在测试集上评估模型
        accuracy, auc, precision, recall = evaluate_model(model, test_loader, criterion)
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Accuracy: {accuracy}, AUC: {auc}, Precision: {precision}, Recall: {recall}')
# 示例参数
vocab_size = 10000  # 假设词汇表大小为10000
embedding_dim = 300  # 假设词嵌入维度为300
hidden_dim = 128  # LSTM隐藏层维度
output_dim = 1  # 输出维度，二分类问题

model = MultiModalModel(vocab_size, embedding_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()


# 训练模型
train_model(model, train_loader, test_loader, optimizer, criterion)

D:\qxy\software-ide\ana\envs\torch-cpu\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1, Loss: 0.14199791436916903, Accuracy: 0.9781904287138584, AUC: 0.5, Precision: 0.0, Recall: 0.0


D:\qxy\software-ide\ana\envs\torch-cpu\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2, Loss: 0.09481779521886717, Accuracy: 0.9781904287138584, AUC: 0.5, Precision: 0.0, Recall: 0.0


D:\qxy\software-ide\ana\envs\torch-cpu\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch 5, Loss: 0.08153771552426076, Accuracy: 0.9781904287138584, AUC: 0.5, Precision: 0.0, Recall: 0.0


D:\qxy\software-ide\ana\envs\torch-cpu\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 6, Loss: 0.07871643060611354, Accuracy: 0.9781904287138584, AUC: 0.5, Precision: 0.0, Recall: 0.0


D:\qxy\software-ide\ana\envs\torch-cpu\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 7, Loss: 0.0777384458349375, Accuracy: 0.9781904287138584, AUC: 0.5, Precision: 0.0, Recall: 0.0
